<a href="https://colab.research.google.com/github/min-hee-Lee/python_workdemo/blob/master/p306_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%91_%EB%84%A4%EC%9D%B4%EB%B2%84%EC%A6%9D%EA%B6%8C_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/python/data

Mounted at /content/drive
/content/drive/MyDrive/python/data


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.NhJDhz0r2U/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.3f7FEHqyoQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.HoRpJk34oU/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [ ]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

#시간 대기 작업할 때 필요함
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
from pickle import bytes_types
import openpyxl

#엑셀 파일 생성
wb = openpyxl.Workbook()

#기본적으로 첫 번째 빈 시트가 생김
#Sheet 워크시트 삭제
wb.remove_sheet(wb['Sheet'])

url = "https://www.naver.com/" 
driver.get(url)

stock = driver.find_element(By.CSS_SELECTOR, '#NM_FAVORITE > div.group_nav > ul.list_nav.NM_FAVORITE_LIST > li:nth-child(3) > a')
print(stock.text)
stock.click()

for i in range(1,5) :
    stock_name = driver.find_element(By.CSS_SELECTOR, '#content > div.article > div.section > div.section_sise_top > ul > li.tab'+str(i)+' > a > span')
    stock_name.click()

    #워크시트 생성 (index=1 가장 최근에 생성된 워크시트가 제일 앞으로 배치됨)
    ws = wb.create_sheet(stock_name.text)  
    ws['A1']='종목'
    ws['B1']='현재가'
    ws['C1']='전일비'
    ws['D1']='등락률' 

    div_stock_list = driver.find_elements(By.CSS_SELECTOR, '#_topItems'+str(i)+' > tr')


    for idx, element_stock in enumerate(div_stock_list, start=1):

        stock_name = driver.find_element(By.CSS_SELECTOR, '#_topItems'+str(i)+' > tr:nth-child('+str(idx)+') > th > a')
        stock_present = driver.find_element(By.CSS_SELECTOR, '#_topItems'+str(i)+' > tr:nth-child('+str(idx)+') > td:nth-child(2)')
        stock_p_day = driver.find_element(By.CSS_SELECTOR, '#_topItems'+str(i)+' > tr:nth-child('+str(idx)+') > td:nth-child(3)')
        stock_rate = driver.find_element(By.CSS_SELECTOR, '#_topItems'+str(i)+' > tr:nth-child('+str(idx)+') > td:nth-child(4)')
        print(stock_name.text, stock_present.text, stock_p_day.text, stock_rate.text)

        ws['A'+str(idx+1)]= stock_name.text  
        ws['B'+str(idx+1)]= stock_present.text
        ws['C'+str(idx+1)]= stock_p_day.text
        ws['D'+str(idx+1)]= stock_rate.text

wb.save('./stock.xlsx')

<ipython-input-11-dddf7bebd687>:9: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  wb.remove_sheet(wb['Sheet'])


증권
MDS테크 3,455 상승
790 +29.64%
KODEX 200선물인버스2X 2,910 상승
55 +1.93%
모아데이타 4,430 상승
190 +4.48%
이수페타시스 9,420 상승
1,290 +15.87%
KODEX 코스닥150선물인버스 4,775 상승
50 +1.06%
스튜디오산타클로스 830 상승
64 +8.36%
KODEX 인버스 4,790 상승
40 +0.84%
에이디칩스 676 상승
16 +2.42%
KODEX 레버리지 14,945 하락
280 -1.84%
실리콘투 3,515 상승
265 +8.15%
아이윈 2,505 상승
235 +10.35%
셀바스AI 29,750 상승
1,050 +3.66%
제주반도체 4,980 상승
150 +3.11%
브리지텍 11,590 상승
2,670 +29.93%
KODEX 코스닥150레버리지 8,125 하락
150 -1.81%
에프앤리퍼블릭 2,405 상한가
555 +30.00%
브리지텍 11,590 상한가
2,670 +29.93%
셀바스헬스케어 6,700 상한가
1,540 +29.84%
MDS테크 3,455 상승
790 +29.64%
뷰노 14,770 상승
3,010 +25.60%
딥노이드 12,610 상승
2,440 +23.99%
라이프시맨틱스 6,860 상승
1,310 +23.60%
코난테크놀로지 147,700 상승
27,200 +22.57%
모비스 2,145 상승
350 +19.50%
아이티아이즈 13,310 상승
2,130 +19.05%
제이엘케이 5,700 상승
860 +17.77%
이수페타시스 9,420 상승
1,290 +15.87%
삼성 레버리지 천연가스 선물 ETN B 2,380 상승
315 +15.25%
대신 S&P 2X 천연가스 선물 ETN 1,905 상승
250 +15.11%
이씨에스 4,480 상승
555 +14.14%
미래에셋비전스팩2호 2,800 하한가
1,200 -30.00%
디와이디 1,330 하한가
570 -30.00%
미래에셋 S&P -2X 천연가스 선물 ETN(H) 64,4